In [ ]:
from scipy.sparse import csr_matrix
from fuzzywuzzy import fuzzy
import pandas as pd
import gc
from sklearn.neighbors import NearestNeighbors

In [ ]:
movie_name = 'God Father'
top_rec_count=5
df_mv = pd.read_csv('./Data Sets/movies.csv',usecols=['movieId', 'title'],dtype={'movieId': 'int32', 'title': 'str'})
df_rt = pd.read_csv('./Data Sets/ratings.csv',usecols=['userId', 'movieId', 'rating'],dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})
model = NearestNeighbors(n_neighbors = 15,algorithm = 'brute',metric = 'cosine')

In [ ]:

matrix = df_rt.pivot(index='movieId', columns='userId', values='rating').fillna(0)
print(matrix)
map = {val: ind for ind, val in enumerate(list(df_mov.set_index('movieId').loc[matrix.index].title)) }
print(map)
matrix = csr_matrix(matrix.values)
print('csr matrix -----')
print(matrix)

In [ ]:
def predict(model, input):
        model.fit(input)
        curr_ratio = 0
        movie_id = None
        for key, value in map.items():
           rat =  fuzzy.ratio(key.lower(), movie_name.lower())
           if(rat>65 & rat >curr_ratio):
               curr_ratio = rat
               movie_id = value
        print(movie_id)
        dist, indexes = model.kneighbors(input[movie_id],n_neighbors=top_rec_count+1)
        result = sorted(list(zip(dist.squeeze().tolist(),indexes.squeeze().tolist())),key=lambda x: x[0])[:0:-1]
        return result

In [ ]:

recommends = predict(model, matrix)
map = {value: key for key, value in map.items()}
print('Recommendation for movie: '+ movie_name)
print(recommends)
for (dist, movie_id) in enumerate(recommends):
    print("movie = "+ map[movie_id])